# Neural Septuple

In [2]:
import numpy as np
import torch

In [253]:
class neural_septuple:

  def __init__(self):

    self.bias = True
    
    self.num_input_vec = 1
    self.num_output_vec = 1
    self.hidden_layer = 2
    self.num_hidden_layer_1_vec = 2
    self.num_hidden_layer_2_vec = 1
    self.num_total_vec = self.num_input_vec + self.num_output_vec + self.num_hidden_layer_1_vec + self.num_hidden_layer_2_vec
    self.idx_layer = [1,2,1,1]
    self.state_vec = torch.tensor([100,0,0,0,200]).unsqueeze(dim=0).T
    self.vec_idx = torch.tensor([i for i in range(len(self.state_vec))])
    
    p_in = torch.ones((self.num_input_vec, 1))
    p_out = torch.ones((self.num_output_vec, 1))

    initial_p_in = torch.zeros((self.num_total_vec,1))
    initial_p_in[:self.num_input_vec] = p_in
    initial_p_out = torch.zeros((self.num_total_vec,1))
    initial_p_out[-self.num_output_vec:] = p_out

    self.p_in = self.create_projection_matrix(initial_p_in)
    self.p_out = self.create_projection_matrix(initial_p_out)
    I = torch.eye(self.num_total_vec)
    self.p_bulk = I - (self.p_in + self.p_out)
    self.p = self.p_in + self.p_out

    self.input_vec = self.p_in @ self.state_vec.type(torch.float)
    self.output_vec = self.p_out @ self.state_vec.type(torch.float)

  def create_weight_matrix(self):
    count = 0
    self.final = []
    for i in self.idx_layer:
      l = []
      for j in range(count, count+i):
        l.append(self.vec_idx[j])
      count += i
      self.final.append(l)
    self.weights = torch.zeros((self.num_total_vec, self.num_total_vec))
    i, j = 0,1
    while j < len(self.final):
      for row in self.final[i]:
        for col in self.final[j]:
          self.weights[col, row] = torch.ones((1,1))
      i += 1
      j += 1
    #self.weights[:, -1] = torch.randn((1,1))
    self.bias = torch.zeros((self.num_total_vec, 1))
    for i in range(self.num_input_vec, self.num_total_vec-self.num_output_vec):
        self.bias[i,0] = torch.ones((1,1))
    self.weights.requires_grad = True
    self.bias.requires_grad = True
    return self.weights, self.bias



  def feed_forword(self, x):
    first = x
    self.weights.requires_grad = True
    self.bias.requires_grad = True
    for i in range(self.hidden_layer):
      x = torch.tanh((self.weights @ x) + self.bias) + first
      #print(x)
    x = (self.weights @ x) + self.bias + first
    return x 
      


  def backword_feed(self,x):
    error = (x - self.state_vec).T @ self.p @ (x - self.state_vec)
    print('LOSS: ', error)
    error.backward()
    for i in range(self.num_total_vec):
      for j in range(self.num_total_vec):
        if not self.weights[i,j]:
          self.weights.grad[i,j] = 0
    #print(self.weights.grad, '####')
    with torch.no_grad():
      self.weights = self.weights - (0.01 * self.weights.grad)
      self.bias = self.bias - (0.01 * self.bias.grad)
    #print(self.weights.grad, '****')
    if self.weights.grad:
      self.weights.grad.zero_()
    if self.bias.grad:
      self.bias.grad.zero_()


  def loss(self, pred, actual):
    return (pred - actual).T @ self.p_in @ (pred - actual)
        
    

  def create_projection_matrix(self, x):
    A = np.linalg.inv(x.T @ x)
    return x @ A @ x.T

In [254]:
a = neural_septuple()
a.create_weight_matrix()

(tensor([[0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0.],
         [0., 1., 1., 0., 0.],
         [0., 0., 0., 1., 0.]], requires_grad=True), tensor([[0.],
         [1.],
         [1.],
         [1.],
         [0.]], requires_grad=True))

In [255]:
for i in range(1000):
  pred = a.feed_forword(a.input_vec)
  print('PREDICTION: ', pred)
  a.backword_feed(pred)

Streaming output truncated to the last 5000 lines.
        [199.7730]], grad_fn=<AddBackward0>)
LOSS:  tensor([[0.0515]], grad_fn=<MmBackward0>)
PREDICTION:  tensor([[100.0000],
        [101.0000],
        [101.0000],
        [  5.8604],
        [199.7821]], grad_fn=<AddBackward0>)
LOSS:  tensor([[0.0475]], grad_fn=<MmBackward0>)
PREDICTION:  tensor([[100.0000],
        [101.0000],
        [101.0000],
        [  5.8604],
        [199.7908]], grad_fn=<AddBackward0>)
LOSS:  tensor([[0.0438]], grad_fn=<MmBackward0>)
PREDICTION:  tensor([[100.0000],
        [101.0000],
        [101.0000],
        [  5.8604],
        [199.7992]], grad_fn=<AddBackward0>)
LOSS:  tensor([[0.0403]], grad_fn=<MmBackward0>)
PREDICTION:  tensor([[100.0000],
        [101.0000],
        [101.0000],
        [  5.8605],
        [199.8072]], grad_fn=<AddBackward0>)
LOSS:  tensor([[0.0372]], grad_fn=<MmBackward0>)
PREDICTION:  tensor([[100.0000],
        [101.0000],
        [101.0000],
        [  5.8605],
        [199.8

In [256]:
a.weights

tensor([[  0.0000,   0.0000,   0.0000,   0.0000,   0.0000],
        [  1.0000,   0.0000,   0.0000,   0.0000,   0.0000],
        [  1.0000,   0.0000,   0.0000,   0.0000,   0.0000],
        [  0.0000,   1.9538,   1.9538,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000, 100.4745,   0.0000]])

In [257]:
(a.weights @ a.weights) @ (a.weights @ a.weights)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [258]:
a.p_bulk

tensor([[0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0.]])

In [259]:
a.p_in

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [260]:
a.p_out

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1.]])